In [1]:
import pandas as pd
import numpy as np

In [103]:
icfes_f = pd.read_csv("D:/UARIV_backup/icfes/RUV_SIMAT_ICFES_FINAL_v2.dsv", sep= ";", encoding= "latin-1", low_memory=False) 

In [4]:
icfes_f["VICTIMA"].value_counts() ## antes

0    450055
1     91435
Name: VICTIMA, dtype: int64

In [343]:
icfes_f["COLE_DEPTO_UBICACION"].value_counts() #despues

In [ ]:
icfes_f["VICTIMA"].value_counts() #despues

In [294]:
icfes_f.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541683 entries, 0 to 541682
Data columns (total 45 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   ESTU_PRIMERNOMBRE            541683 non-null  object 
 1   ESTU_SEGUNDONOMBRE           436279 non-null  object 
 2   ESTU_PRIMERAPELLIDO          541683 non-null  object 
 3   ESTU_SEGUNDOAPELLIDO         538923 non-null  object 
 4   ESTU_TIPODOCUMENTO           541683 non-null  object 
 5   ESTU_DOCUMENTO               541683 non-null  object 
 6   ESTU_FECHANACIMIENTO         541683 non-null  object 
 7   ESTU_GENERO                  541589 non-null  object 
 8   ESTU_DEPTO_RESIDE            541522 non-null  object 
 9   ESTU_MCPIO_RESIDE            541522 non-null  object 
 10  COLE_NOMBRE_ESTABLECIMIENTO  541683 non-null  object 
 11  COLE_CALENDARIO              541683 non-null  object 
 12  COLE_GENERO                  541683 non-null  object 
 13 

### General

In [176]:
grl= icfes_f.groupby(by=["VICTIMA"]).aggregate({"PUNT_GLOBAL": [np.mean, np.size, np.var]})#, np.max, np.min, np.mean, np.std]})
grl
#np.mean, np.var(ddof=1), np.size

# grl_p= icfes_f.pivot_table(values= ["PUNT_GLOBAL"],
#                                   index=["VICTIMA"],#, "sexo"],
#                                 aggfunc= ["mean", "count", "var(ddof=1)"])#, margins = True
# agregar nueva fila a dataframe mediante listasgrl_p.loc[2] = [1, 1,1] 

PUNT_GLOBAL                     
               mean    size          var
VICTIMA                                 
0        249.760499  449978  2605.475291
1        226.697508   91705  1921.956631

In [177]:
grl = grl.stack()
grl = grl.unstack(0)

In [111]:
grl.columns

MultiIndex([('PUNT_GLOBAL', 0),
            ('PUNT_GLOBAL', 1)],
           names=[None, 'VICTIMA'])

In [178]:
grl["dif_mean"]  = grl[('PUNT_GLOBAL', 0)] - grl[('PUNT_GLOBAL', 1)]

In [179]:
#s = np.sqrt((((91435 - 1)* var_v) + ((450055 - 1) * var_nv)) / (91435 + 450055 - 2))
#g = n/s

In [120]:
grl.index

Index(['mean', 'size', 'var'], dtype='object')

In [180]:
g = grl.loc["mean",('dif_mean', '')] / np.sqrt((((grl.loc["size", ('PUNT_GLOBAL', 1)] - 1)* grl.loc["var",('PUNT_GLOBAL', 1)]) + ((grl.loc["size",('PUNT_GLOBAL', 0)] - 1) * grl.loc["var",('PUNT_GLOBAL', 0)])) / (grl.loc["size",('PUNT_GLOBAL', 1)] + grl.loc["size",('PUNT_GLOBAL', 0)] - 2))
g

0.46220750015692447

# Sexo

In [181]:
# grl_s= icfes_f.pivot_table(values= ["PUNT_GLOBAL"],
#                                   index=["ESTU_GENERO"],#, "sexo"],
#                                   columns=["VICTIMA"], aggfunc= ["mean", "count", "var"], margins = True)
# grl_s
grl_s= icfes_f.groupby(by=["VICTIMA", "ESTU_GENERO"]).aggregate({"PUNT_GLOBAL": [np.mean, np.size, np.var]})#, np.max, np.min, np.mean, np.std]})
grl_s

PUNT_GLOBAL                     
                           mean    size          var
VICTIMA ESTU_GENERO                                 
0       F            246.388729  242256  2462.612225
        M            253.694529  207640  2743.719593
1       F            224.369403   52620  1755.909074
        M            229.825097   39073  2128.664996

In [182]:
grl_s = grl_s.stack()
grl_s = grl_s.unstack(0)
grl_s

PUNT_GLOBAL              
VICTIMA                       0             1
ESTU_GENERO                                  
F           mean     246.388729    224.369403
            size  242256.000000  52620.000000
            var     2462.612225   1755.909074
M           mean     253.694529    229.825097
            size  207640.000000  39073.000000
            var     2743.719593   2128.664996

In [142]:
grl_s.index

MultiIndex([('F', 'mean'),
            ('F', 'size'),
            ('F',  'var'),
            ('M', 'mean'),
            ('M', 'size'),
            ('M',  'var')],
           names=['ESTU_GENERO', None])

In [147]:
grl_s.columns

MultiIndex([('PUNT_GLOBAL',  0),
            ('PUNT_GLOBAL',  1),
            (   'dif_mean', '')],
           names=[None, 'VICTIMA'])

In [183]:
##crear numerador
grl_s["dif_mean"]  = grl_s[('PUNT_GLOBAL', 0)] - grl_s[('PUNT_GLOBAL', 1)]

In [184]:
#prueba
for i in l:
    print(grl_s.loc[(i,'size'),('PUNT_GLOBAL', 0)])

242256.0
207640.0


In [185]:
###calcular g
l1 = ['F', 'M'] 

for i in l1:
    exec("g_{} = grl_s.loc[(i, 'mean'),('dif_mean', '')] / np.sqrt((((grl_s.loc[(i,'size'), ('PUNT_GLOBAL', 1)] - 1)* grl_s.loc[(i,'var'),('PUNT_GLOBAL', 1)]) + ((grl_s.loc[(i,'size'),('PUNT_GLOBAL', 0)] - 1) * grl_s.loc[(i,'var'),('PUNT_GLOBAL', 0)])) / (grl_s.loc[(i,'size'),('PUNT_GLOBAL', 1)] + grl_s.loc[(i,'size'),('PUNT_GLOBAL', 0)] - 2))".format(i))
    exec("print(i + ' ' + str(g_{}))".format(i))

F 0.45553395004376485
M 0.46400379366937916


# Grupo etnico

In [187]:
# grl_s= icfes_f.pivot_table(values= ["PUNT_GLOBAL"],
#                                   index=["ESTU_GENERO"],#, "sexo"],
#                                   columns=["VICTIMA"], aggfunc= ["mean", "count", "var"], margins = True)
# grl_s
grl_e= icfes_f.groupby(by=["VICTIMA", "GRUPO_ETCNI"]).aggregate({"PUNT_GLOBAL": [np.mean, np.size, np.var]})#, np.max, np.min, np.mean, np.std]})
grl_e

PUNT_GLOBAL                      
                           mean    size           var
VICTIMA GRUPO_ETCNI                                  
0       0.0          252.615404  412617   2562.410709
        1.0          213.418259   13133   1916.118299
        2.0          215.138289    8193   1851.528115
        3.0          283.500000       2     40.500000
        4.0          225.227273      22   2402.564935
        5.0          223.311224     196   1862.430848
        6.0          223.107883    6952   1873.887942
        7.0          257.333333       6   2304.666667
1       0.0          230.699022   76205   1918.284273
        1.0          206.636779    3725   1705.279152
        2.0          204.580228    7354   1377.941951
        3.0          187.000000       3    921.000000
        4.0          245.500000       2  14620.500000
        5.0          243.166667      12   1330.696970
        6.0          211.310313    4402   1405.199753
        7.0          256.500000       2   2964.500000

In [188]:
grl_e = grl_e.stack()
grl_e = grl_e.unstack(0)
grl_e

PUNT_GLOBAL              
VICTIMA                       0             1
GRUPO_ETCNI                                  
0.0         mean     252.615404    230.699022
            size  412617.000000  76205.000000
            var     2562.410709   1918.284273
1.0         mean     213.418259    206.636779
            size   13133.000000   3725.000000
            var     1916.118299   1705.279152
2.0         mean     215.138289    204.580228
            size    8193.000000   7354.000000
            var     1851.528115   1377.941951
3.0         mean     283.500000    187.000000
            size       2.000000      3.000000
            var       40.500000    921.000000
4.0         mean     225.227273    245.500000
            size      22.000000      2.000000
            var     2402.564935  14620.500000
5.0         mean     223.311224    243.166667
            size     196.000000     12.000000
            var     1862.430848   1330.696970
6.0         mean     223.107883    211.310313
            size    6952.000000   4402.000000
            var     1873.887942   1405.199753
7.0         mean     257.333333    256.500000
            size       6.000000      2.000000
            var     2304.666667   2964.500000

In [289]:
##crear numerador
grl_e["dif_mean"]  = grl_e[('PUNT_GLOBAL', 0)] - grl_e[('PUNT_GLOBAL', 1)]

PUNT_GLOBAL                     dif_mean
VICTIMA                       0             1               
GRUPO_ETCNI                                                 
0.0         mean     252.615404    230.699022      21.916381
            size  412617.000000  76205.000000  336412.000000
            var     2562.410709   1918.284273     644.126435
1.0         mean     213.418259    206.636779       6.781481
            size   13133.000000   3725.000000    9408.000000
            var     1916.118299   1705.279152     210.839147
2.0         mean     215.138289    204.580228      10.558060
            size    8193.000000   7354.000000     839.000000
            var     1851.528115   1377.941951     473.586164
3.0         mean     283.500000    187.000000      96.500000
            size       2.000000      3.000000      -1.000000
            var       40.500000    921.000000    -880.500000
4.0         mean     225.227273    245.500000     -20.272727
            size      22.000000      2.000000      20.000000
            var     2402.564935  14620.500000  -12217.935065
5.0         mean     223.311224    243.166667     -19.855442
            size     196.000000     12.000000     184.000000
            var     1862.430848   1330.696970     531.733878
6.0         mean     223.107883    211.310313      11.797569
            size    6952.000000   4402.000000    2550.000000
            var     1873.887942   1405.199753     468.688190
7.0         mean     257.333333    256.500000       0.833333
            size       6.000000      2.000000       4.000000
            var     2304.666667   2964.500000    -659.833333

In [291]:
grl_e.loc[(0.0, 'mean'),('dif_mean', '')]

21.916381257110146

In [284]:
#grl_e.loc[(0,'mean')]
#lp= grl_e.index.to_list()
#grl_e.loc[(0.0, 'mean'):(7, 'mean')]
# for i in enumerate(grl_e.index):
#     print(i)
#print(grl_s.loc[(i,'size'),('PUNT_GLOBAL', 0)])

In [292]:
###calcular g
l2 = [0,1,2,3,4,5,6,7] 

for i in l2:
    exec("g_{} = grl_e.loc[(i, 'mean'),('dif_mean', '')] / np.sqrt((((grl_e.loc[(i,'size'), ('PUNT_GLOBAL', 1)] - 1)* grl_e.loc[(i,'var'),('PUNT_GLOBAL', 1)]) + ((grl_e.loc[(i,'size'),('PUNT_GLOBAL', 0)] - 1) * grl_e.loc[(i,'var'),('PUNT_GLOBAL', 0)])) / (grl_e.loc[(i,'size'),('PUNT_GLOBAL', 1)] + grl_e.loc[(i,'size'),('PUNT_GLOBAL', 0)] - 2))".format(i))
    exec("print(str(i) + ' ' + str(g_{}))".format(i))

0 0.44169779189045366
1 0.15684021931888534
2 0.2617107718444759
3 3.8523030830692306
4 -0.3727507735442488
5 -0.4636342192259956
6 0.2867930785958952
7 0.016958703942181512


## Grado

In [297]:
grl_g= icfes_f.groupby(by=["VICTIMA", "COLE_GRADO"]).aggregate({"PUNT_GLOBAL": [np.mean, np.size, np.var]})#, np.max, np.min, np.mean, np.std]})
grl_g

PUNT_GLOBAL                     
                          mean    size          var
VICTIMA COLE_GRADO                                 
0       11          253.641741  405589  2559.837423
        26          214.297011   44389  1627.209611
1       11          229.771448   80870  1931.751508
        26          203.754315   10835  1252.050211

In [298]:
grl_g = grl_g.stack()
grl_g = grl_g.unstack(0)
grl_g

PUNT_GLOBAL              
VICTIMA                      0             1
COLE_GRADO                                  
11         mean     253.641741    229.771448
           size  405589.000000  80870.000000
           var     2559.837423   1931.751508
26         mean     214.297011    203.754315
           size   44389.000000  10835.000000
           var     1627.209611   1252.050211

In [299]:
##crear numerador
grl_g["dif_mean"]  = grl_g[('PUNT_GLOBAL', 0)] - grl_g[('PUNT_GLOBAL', 1)]

In [312]:
###calcular g
l3 = [11,26] 

for i in l3:
    exec("g_{} = grl_g.loc[(i, 'mean'),('dif_mean', '')] / np.sqrt((((grl_g.loc[(i,'size'), ('PUNT_GLOBAL', 1)] - 1)* grl_g.loc[(i,'var'),('PUNT_GLOBAL', 1)]) + ((grl_g.loc[(i,'size'),('PUNT_GLOBAL', 0)] - 1) * grl_g.loc[(i,'var'),('PUNT_GLOBAL', 0)])) / (grl_g.loc[(i,'size'),('PUNT_GLOBAL', 1)] + grl_g.loc[(i,'size'),('PUNT_GLOBAL', 0)] - 2))".format(i))
    exec("print(str(i) + ' ' + str(g_{}))".format(i))

11 0.48171980810127846
26 0.26747369330579807


## Naturaleza del colegio

In [304]:
grl_n= icfes_f.groupby(by=["VICTIMA", "COLE_NATURALEZA"]).aggregate({"PUNT_GLOBAL": [np.mean, np.size, np.var]})#, np.max, np.min, np.mean, np.std]})
grl_n

PUNT_GLOBAL                     
                               mean    size          var
VICTIMA COLE_NATURALEZA                                 
0       NO OFICIAL       272.800632  112069  3073.131371
        OFICIAL          242.119139  337909  2215.934686
1       NO OFICIAL       233.043243    8510  2483.935608
        OFICIAL          226.048404   83195  1859.960647

In [305]:
grl_n = grl_n.stack()
grl_n = grl_n.unstack(0)
grl_n

PUNT_GLOBAL              
VICTIMA                           0             1
COLE_NATURALEZA                                  
NO OFICIAL      mean     272.800632    233.043243
                size  112069.000000   8510.000000
                var     3073.131371   2483.935608
OFICIAL         mean     242.119139    226.048404
                size  337909.000000  83195.000000
                var     2215.934686   1859.960647

In [310]:
grl_n.index

MultiIndex([('NO OFICIAL', 'mean'),
            ('NO OFICIAL', 'size'),
            ('NO OFICIAL',  'var'),
            (   'OFICIAL', 'mean'),
            (   'OFICIAL', 'size'),
            (   'OFICIAL',  'var')],
           names=['COLE_NATURALEZA', None])

In [311]:
##crear numerador
grl_n["dif_mean"]  = grl_n[('PUNT_GLOBAL', 0)] - grl_n[('PUNT_GLOBAL', 1)]

In [317]:
grl_n.loc[('NO OFICIAL', 'mean'),('dif_mean', '')]

39.75738851040853

In [318]:
###calcular g
l4 = ['NO OFICIAL', 'OFICIAL'] 
l5 = ['NO_OFICIAL', 'OFICIAL']

for i,j in zip(l4,l5):
    exec("g_{} = grl_n.loc[(i, 'mean'),('dif_mean', '')] / np.sqrt((((grl_n.loc[(i,'size'), ('PUNT_GLOBAL', 1)] - 1)* grl_n.loc[(i,'var'),('PUNT_GLOBAL', 1)]) + ((grl_n.loc[(i,'size'),('PUNT_GLOBAL', 0)] - 1) * grl_n.loc[(i,'var'),('PUNT_GLOBAL', 0)])) / (grl_n.loc[(i,'size'),('PUNT_GLOBAL', 1)] + grl_n.loc[(i,'size'),('PUNT_GLOBAL', 0)] - 2))".format(j))
    exec("print(j + ' ' + str(g_{}))".format(j))

NO_OFICIAL 0.7220799889909494
OFICIAL 0.34694495471947184


In [ ]:
## gENERO DEL COLEGIO

In [328]:
grl_gro= icfes_f.groupby(by=["VICTIMA", "COLE_GENERO"]).aggregate({"PUNT_GLOBAL": [np.mean, np.size, np.var]})#, np.max, np.min, np.mean, np.std]})
grl_gro

PUNT_GLOBAL                     
                           mean    size          var
VICTIMA COLE_GENERO                                 
0       FEMENINO     282.179975   11835  2241.113119
        MASCULINO    296.932464    3835  2846.827206
        MIXTO        248.460526  434308  2563.305372
1       FEMENINO     242.372800    1250  2274.475801
        MASCULINO    227.011070     271  1864.714692
        MIXTO        226.479298   90184  1913.834684

In [329]:
grl_gro = grl_gro.stack()
grl_gro = grl_gro.unstack(0)
grl_gro

PUNT_GLOBAL              
VICTIMA                       0             1
COLE_GENERO                                  
FEMENINO    mean     282.179975    242.372800
            size   11835.000000   1250.000000
            var     2241.113119   2274.475801
MASCULINO   mean     296.932464    227.011070
            size    3835.000000    271.000000
            var     2846.827206   1864.714692
MIXTO       mean     248.460526    226.479298
            size  434308.000000  90184.000000
            var     2563.305372   1913.834684

In [330]:
##crear numerador
grl_gro["dif_mean"]  = grl_gro[('PUNT_GLOBAL', 0)] - grl_gro[('PUNT_GLOBAL', 1)]

In [331]:
###calcular g
l7 = ['FEMENINO', 'MASCULINO', 'MIXTO'] 

for i in l7:
    exec("g_{} = grl_gro.loc[(i, 'mean'),('dif_mean', '')] / np.sqrt((((grl_gro.loc[(i,'size'), ('PUNT_GLOBAL', 1)] - 1)* grl_gro.loc[(i,'var'),('PUNT_GLOBAL', 1)]) + ((grl_gro.loc[(i,'size'),('PUNT_GLOBAL', 0)] - 1) * grl_gro.loc[(i,'var'),('PUNT_GLOBAL', 0)])) / (grl_gro.loc[(i,'size'),('PUNT_GLOBAL', 1)] + grl_gro.loc[(i,'size'),('PUNT_GLOBAL', 0)] - 2))".format(i))
    exec("print(i + ' ' + str(g_{}))".format(i))

FEMENINO 0.8402742923689344
MASCULINO 1.3256069436753226
MIXTO 0.44393996915130474


## Discapacidad física

In [322]:
grl_df= icfes_f.groupby(by=["VICTIMA", "ESTU_DISC_FISICA"]).aggregate({"PUNT_GLOBAL": [np.mean, np.size, np.var]})#, np.max, np.min, np.mean, np.std]})
grl_df

PUNT_GLOBAL                  
                                mean size          var
VICTIMA ESTU_DISC_FISICA                              
0       x                 229.786307  482  2606.201642
1       x                 220.929412   85  1935.137815

In [323]:
grl_df = grl_df.stack()
grl_df = grl_df.unstack(0)
grl_df

PUNT_GLOBAL             
VICTIMA                          0            1
ESTU_DISC_FISICA                               
x                mean   229.786307   220.929412
                 size   482.000000    85.000000
                 var   2606.201642  1935.137815

In [324]:
##crear numerador
grl_df["dif_mean"]  = grl_df[('PUNT_GLOBAL', 0)] - grl_df[('PUNT_GLOBAL', 1)]

In [326]:
###calcular g
l6 = ['x'] 

for i in l6:
    exec("g_{} = grl_df.loc[(i, 'mean'),('dif_mean', '')] / np.sqrt((((grl_df.loc[(i,'size'), ('PUNT_GLOBAL', 1)] - 1)* grl_df.loc[(i,'var'),('PUNT_GLOBAL', 1)]) + ((grl_df.loc[(i,'size'),('PUNT_GLOBAL', 0)] - 1) * grl_df.loc[(i,'var'),('PUNT_GLOBAL', 0)])) / (grl_df.loc[(i,'size'),('PUNT_GLOBAL', 1)] + grl_df.loc[(i,'size'),('PUNT_GLOBAL', 0)] - 2))".format(i))
    exec("print(i + ' ' + str(g_{}))".format(i))

x 0.17691044397764238


## Zona de ubicación

In [337]:
grl_z= icfes_f.groupby(by=["VICTIMA", "COLE_AREA_UBICACION"]).aggregate({"PUNT_GLOBAL": [np.mean, np.size, np.var]})#, np.max, np.min, np.mean, np.std]})
grl_z

PUNT_GLOBAL                     
                                   mean    size          var
VICTIMA COLE_AREA_UBICACION                                 
0       RURAL                227.802327   67384  2226.003487
        URBANO               253.627861  382594  2572.438439
1       RURAL                214.248357   22669  1593.642039
        URBANO               230.785373   69036  1962.186208

In [338]:
grl_z = grl_z.stack()
grl_z = grl_z.unstack(0)
grl_z

PUNT_GLOBAL              
VICTIMA                               0             1
COLE_AREA_UBICACION                                  
RURAL               mean     227.802327    214.248357
                    size   67384.000000  22669.000000
                    var     2226.003487   1593.642039
URBANO              mean     253.627861    230.785373
                    size  382594.000000  69036.000000
                    var     2572.438439   1962.186208

In [339]:
##crear numerador
grl_z["dif_mean"]  = grl_z[('PUNT_GLOBAL', 0)] - grl_z[('PUNT_GLOBAL', 1)]

In [340]:
###calcular g
l7 = ['URBANO', 'RURAL'] 

for i in l7:
    exec("g_{} = grl_z.loc[(i, 'mean'),('dif_mean', '')] / np.sqrt((((grl_z.loc[(i,'size'), ('PUNT_GLOBAL', 1)] - 1)* grl_z.loc[(i,'var'),('PUNT_GLOBAL', 1)]) + ((grl_z.loc[(i,'size'),('PUNT_GLOBAL', 0)] - 1) * grl_z.loc[(i,'var'),('PUNT_GLOBAL', 0)])) / (grl_z.loc[(i,'size'),('PUNT_GLOBAL', 1)] + grl_z.loc[(i,'size'),('PUNT_GLOBAL', 0)] - 2))".format(i))
    exec("print(i + ' ' + str(g_{}))".format(i))

URBANO 0.45876623810427797
RURAL 0.2981363232056293


## Departamentos

In [104]:
### eliminar espacios blanco para cada departamentos
icfes_f["COLE_DEPTO_UBICACION_TRIM"]= icfes_f["COLE_DEPTO_UBICACION"].replace(' ', '', regex =True)

In [105]:
grl_dpto= icfes_f.groupby(by=["VICTIMA", "COLE_DEPTO_UBICACION_TRIM"]).aggregate({"PUNT_GLOBAL": [np.mean, np.size, np.var]})#, np.max, np.min, np.mean, np.std]})
grl_dpto

PUNT_GLOBAL                    
                                         mean   size          var
VICTIMA COLE_DEPTO_UBICACION_TRIM                                
0       AMAZONAS                   213.053846    650  1910.017127
        ANTIOQUIA                  249.899848  54577  2643.334370
        ARAUCA                     237.966767   1986  2197.395369
        ATLANTICO                  241.602891  29541  2664.536036
        BOGOTÁ                     267.046899  76356  2398.769184
...                                       ...    ...          ...
1       SUCRE                      224.774911   3914  1905.093457
        TOLIMA                     229.227570   2597  1689.904663
        VALLE                      224.968690   6803  1891.360237
        VAUPES                     219.476190     84  1202.083764
        VICHADA                    221.178947     95  1834.701680

[66 rows x 3 columns]

In [106]:
grl_dpto = grl_dpto.stack()
grl_dpto = grl_dpto.unstack(0)
grl_dpto

PUNT_GLOBAL              
VICTIMA                                    0             1
COLE_DEPTO_UBICACION_TRIM                                 
AMAZONAS                  mean    213.053846    223.444444
                          size    650.000000     27.000000
                          var    1910.017127   1701.333333
ANTIOQUIA                 mean    249.899848    225.619085
                          size  54577.000000  18004.000000
...                                      ...           ...
VAUPES                    size    306.000000     84.000000
                          var    1673.713747   1202.083764
VICHADA                   mean    218.510597    221.178947
                          size    519.000000     95.000000
                          var    2134.543806   1834.701680

[99 rows x 2 columns]

In [107]:
##crear numerador
grl_dpto["dif_mean"]  = grl_dpto[('PUNT_GLOBAL', 0)] - grl_dpto[('PUNT_GLOBAL', 1)]

In [108]:
# convertir categorías en listas para recorrerlas
#l8 = icfes_f["COLE_DEPTO_UBICACION"].replace(' ', '', regex =True).value_counts().index.to_list() 
l8 = icfes_f["COLE_DEPTO_UBICACION_TRIM"].value_counts().index.to_list() 


In [109]:
###calcular g
for i in l8:
    exec("g_{} = grl_dpto.loc[(i, 'mean'),('dif_mean', '')] / np.sqrt((((grl_dpto.loc[(i,'size'), ('PUNT_GLOBAL', 1)] - 1)* grl_dpto.loc[(i,'var'),('PUNT_GLOBAL', 1)]) + ((grl_dpto.loc[(i,'size'),('PUNT_GLOBAL', 0)] - 1) * grl_dpto.loc[(i,'var'),('PUNT_GLOBAL', 0)])) / (grl_dpto.loc[(i,'size'),('PUNT_GLOBAL', 1)] + grl_dpto.loc[(i,'size'),('PUNT_GLOBAL', 0)] - 2))".format(i))
    exec("print(i + ' ' + str(g_{}))".format(i))

BOGOTÁ 0.478629018367419
ANTIOQUIA 0.4883353610754436
VALLE 0.47471781616374853
CUNDINAMARCA 0.3905823393300851
ATLANTICO 0.20965606281345078
BOLIVAR 0.1920776712944998
SANTANDER 0.38688713009960685
CORDOBA 0.21201635884200185
NARIÑO 0.6247431665488887
NORTESANTANDER 0.4025686389662108
TOLIMA 0.2702209302790968
MAGDALENA 0.3056832169421139
BOYACA 0.3925147386694157
HUILA 0.22316077297999817
CAUCA 0.1708909361056205
CESAR 0.28040538740691107
META 0.3854732972159484
SUCRE 0.23353169301362428
CALDAS 0.30460589499606266
RISARALDA 0.4520751883954432
LAGUAJIRA 0.030655400231657387
QUINDIO 0.45133742343277944
CASANARE 0.17351591291300525
CHOCO 0.3517287494051003
PUTUMAYO 0.12976777895592403
CAQUETA 0.21492289860375902
ARAUCA 0.12592097043590228
GUAVIARE 0.24099122414205687
AMAZONAS -0.23825263003746266
VICHADA -0.0583884093305097
SANANDRES 0.38237539112664837
VAUPES -0.3633699298886584
GUAINIA -0.23460500986185237


## Municipios > 3 vic y no víctimas

In [110]:
### eliminar espacios blanco, guiones, puntos para cada municipio
icfes_f["COLE_MCPIO_UBICACION_TRIM"]= icfes_f["COLE_MCPIO_UBICACION"].replace("[' '-.]", '', regex =True)
#icfes_f["COLE_MCPIO_UBICACION_TRIM"]= icfes_f["COLE_MCPIO_UBICACION_TRIM"].replace('-', '', regex =True)

In [ ]:
###agrupar para crear nuevas variables a partir de valores de una variable
# data = data.groupby(['type', 'status', 'name'])['value'].agg(
#     [('one',  np.mean), 
#     ('two', lambda value: 100* ((value>32).sum() / reading.mean())), 
#     ('test2', lambda value: 100* ((value > 45).sum() / value.mean()))])

In [111]:
grl_mun= icfes_f.groupby(by=["VICTIMA", "COLE_MCPIO_UBICACION_TRIM"]).aggregate({"PUNT_GLOBAL": [np.mean, np.size, np.var]})
grl_mun

PUNT_GLOBAL                  
                                         mean size          var
VICTIMA COLE_MCPIO_UBICACION_TRIM                              
0       ABEJORRAL                  247.356589  129  2047.824976
        ABRIAQUÍ                   229.400000    5  1419.300000
        ACACÍAS                    253.824499  849  1993.227418
        ACANDÍ                     201.666667   15  1352.952381
        ACEVEDO                    231.200680  294  1641.758225
...                                       ...  ...          ...
1       ZIPAQUIRÁ                  250.500000   30  2043.224138
        ZONABANANERA               200.781818  330  1012.262282
        ÁBREGO                     247.122807   57  1036.359649
        ÍQUIRA                     227.333333   24  2856.753623
        ÚTICA                      250.750000   12  1298.568182

[1999 rows x 3 columns]

In [112]:
msk_1 = grl_mun[('PUNT_GLOBAL', 'size')] > 3
grl_mun1 = grl_mun[msk_1==True]

In [113]:
grl_mun1 = grl_mun1.stack()
grl_mun2 = grl_mun1.unstack(0)
grl_mun2

PUNT_GLOBAL             
VICTIMA                                   0            1
COLE_MCPIO_UBICACION_TRIM                               
ABEJORRAL                 mean   247.356589   241.652174
                          size   129.000000    69.000000
                          var   2047.824976  2385.112532
ABRIAQUÍ                  mean   229.400000   221.600000
                          size     5.000000    10.000000
...                                     ...          ...
HACARÍ                    size          NaN    42.000000
                          var           NaN  1049.879791
RECETOR                   mean          NaN   230.250000
                          size          NaN     8.000000
                          var           NaN   784.500000

[3090 rows x 2 columns]

In [114]:
##crear numerador
grl_mun2["dif_mean"]  = grl_mun2[('PUNT_GLOBAL', 0)] - grl_mun2[('PUNT_GLOBAL', 1)]

In [115]:
##contar nulos
grl_mun2[('PUNT_GLOBAL', 1)].isna().sum()

525

In [116]:
#reemplazar nulos en víctimas
grl_mun2[('PUNT_GLOBAL', 1)] = pd.Series(grl_mun2[('PUNT_GLOBAL', 1)], dtype = "int").replace(np.nan, -1)

In [117]:
#reemplazar nulos en no víctimas víctimas
grl_mun2[('PUNT_GLOBAL', 0)] = pd.Series(grl_mun2[('PUNT_GLOBAL', 0)], dtype = "int").replace(np.nan, -1)

In [118]:
### eliminar nulos
msk_2 = (grl_mun2[('PUNT_GLOBAL', 0)] != -1) & (grl_mun2[('PUNT_GLOBAL', 1)] != -1)
grl_mun3 = grl_mun2[msk_2==True]

In [119]:
##contar nulos
grl_mun3[('PUNT_GLOBAL', 0)].isna().sum()

0

In [120]:
grl_mun3

PUNT_GLOBAL                 dif_mean
VICTIMA                                   0            1            
COLE_MCPIO_UBICACION_TRIM                                           
ABEJORRAL                 mean   247.356589   241.652174    5.704415
                          size   129.000000    69.000000   60.000000
                          var   2047.824976  2385.112532 -337.287556
ABRIAQUÍ                  mean   229.400000   221.600000    7.800000
                          size     5.000000    10.000000   -5.000000
...                                     ...          ...         ...
ÍQUIRA                    size    90.000000    24.000000   66.000000
                          var   2981.657928  2856.753623  124.904304
ÚTICA                     mean   231.837209   250.750000  -18.912791
                          size    43.000000    12.000000   31.000000
                          var    997.330011  1298.568182 -301.238171

[2559 rows x 3 columns]

In [121]:
grl_mun4 = grl_mun3.reset_index()

In [122]:
grl_mun4.columns

MultiIndex([('COLE_MCPIO_UBICACION_TRIM', ''),
            (                  'level_1', ''),
            (              'PUNT_GLOBAL',  0),
            (              'PUNT_GLOBAL',  1),
            (                 'dif_mean', '')],
           names=[None, 'VICTIMA'])

In [123]:
# convertir categorías en listas para recorrerlas
l9 = grl_mun4[('COLE_MCPIO_UBICACION_TRIM', '')].value_counts().index.to_list() 

In [124]:
###calcular g
for i in l9:
    exec("g_{} = grl_mun3.loc[(i, 'mean'),('dif_mean', '')] / np.sqrt((((grl_mun3.loc[(i,'size'), ('PUNT_GLOBAL', 1)] - 1)* grl_mun3.loc[(i,'var'),('PUNT_GLOBAL', 1)]) + ((grl_mun3.loc[(i,'size'),('PUNT_GLOBAL', 0)] - 1) * grl_mun3.loc[(i,'var'),('PUNT_GLOBAL', 0)])) / (grl_mun3.loc[(i,'size'),('PUNT_GLOBAL', 1)] + grl_mun3.loc[(i,'size'),('PUNT_GLOBAL', 0)] - 2))".format(i))
    exec("print(i + ' ' + str(g_{}))".format(i))

ABEJORRAL 0.12260210234987413
PUERTOSANTANDER -0.23973895822601732
PUERTOTRIUNFO 0.04523313669656425
PUERTOWILCHES -0.11899687851116904
PULÍ 0.7718662477180577
PUPIALES 0.26757689342407176
PURACÉ -0.3253651762267519
PURIFICACIÓN 0.11301807455340726
PURÍSIMADELACONCEPCIÓN -0.051057267001094114
PÁCORA 0.8691163929341256
PÁEZ -0.3993508000341331
QUEBRADANEGRA 0.019619456704264963
QUETAME 0.09083721339356578
QUIBDÓ 0.36300509642862033
QUIMBAYA 0.5650095127009029
QUINCHÍA -0.044017472900583665
QUIPILE 0.15564044629498963
RAGONVALIA 1.2661091621179985
RAMIRIQUÍ -0.2698763268611427
REGIDOR 0.6889620654476356
REMEDIOS 0.11945508186140613
REMOLINO -0.14102206921152508
REPELÓN 0.02215806873562722
RESTREPO 0.20019705560345138
RETIRO 0.48757418922107815
PUERTOTEJADA 0.13684291278364452
PUERTOSALGAR 0.24241230866721625
PROVIDENCIA -0.5172795377635188
PUERTORONDÓN 0.05354282685689764
PUEBLONUEVO -0.018994071542600376
PUEBLORICO 0.4010998922251247
PUEBLORRICO 0.07035733731845852
PUEBLOVIEJO -0.099990

YUMBO 0.23143604200676898
ZAMBRANO 0.12774411334456173
ZAPATOCA 0.06911687834860471
ZAPAYÁN 0.12600186453613088
ZARAGOZA 0.13193694987827032
ZARZAL 0.33359335558333736
ZETAQUIRA 0.17761507957488035
ZIPACÓN -0.33896013631548433
ZIPAQUIRÁ 0.49050751054325736
ZONABANANERA 0.09127492172232388
ÁBREGO -0.15566659936584543
ÍQUIRA 0.20027685750769125
VIOTÁ 0.39358726249847314
VILLETA 0.18496370549809352
VILLAVIEJA 0.1052360720533801
VILLACARO -0.16264863277295574
VALLEDESANJUAN 0.2814587953312797
VALLEDUPAR 0.36607653309083077
VALPARAÍSO 0.3908460089578905
VEGACHÍ 0.18999319942964035
VENADILLO 0.10911423446177602
VENECIA 0.06505202936405606
VERGARA 0.45341245709576056
VICTORIA 0.17365529190147674
VIGÍADELFUERTE -0.405683818310603
VIJES 0.12055739166985774
VILLADELEYVA 0.27614397781970507
VILLAVICENCIO 0.4078397185301939
VILLADELROSARIO 0.2227267167708139
VILLADESANDIEGODEUBATÉ 0.4693467831214472
VILLAGARZÓN 0.22153960776278428
VILLAGÓMEZ 0.6391177879081265
VILLAHERMOSA 0.201514521150049
VILLAM

BAGADÓ -0.16259748936807872
BAHÍASOLANO 0.06688249128668217
BAJOBAUDÓ -0.13355027614042908
BALBOA -0.23058696209208843
BARAYA 0.2969856944726632
BELALCÁZAR -0.06900170920775951
BARBACOAS 0.00986041030608381
BARBOSA 0.28306494005304866
BARICHARA 0.6236911177869298
BARRANCABERMEJA 0.5156519527316566
BARRANCADEUPÍA 0.05612654137918328
BARRANCAS 0.11680833806086315
BARRANCODELOBA -0.01214650129431723
BARRANCOMINAS -0.5237935695675723
BARRANQUILLA 0.26744143644912716
BECERRIL 0.07185935288219826
DONMATÍAS -0.14466832753981645
DOSQUEBRADAS 0.37001411943732915
DUITAMA 0.5756457021272229
LABELLEZA 0.1288231474191555
LACALERA 0.19952681661653204
LACEJA 0.30465435371076866
LACELIA 0.22366314715548616
LACRUZ -0.12350063897158839
LACUMBRE 0.10414101790904387
LADORADA 0.1130900179753368
LAESPERANZA -0.04855347082154645
LAESTRELLA 0.11244000812925539
LAFLORIDA 0.498284281325509
LAGLORIA 0.03193506537475017
LAJAGUADEIBIRICO 0.10058165580847538
LAJAGUADELPILAR 0.03855429403965072
LALLANADA -0.06271553